In [ ]:
# reference: https://amarsaini.github.io/Epoching-Blog/jupyter/2020/08/10/Latent-Space-Exploration-with-StyleGAN2.html#5.-Projecting-our-own-Input-Images-into-the-Latent-Space

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 204kB 8.7MB/s 
     |████████████████████████████████| 112kB 16.9MB/s 
     |████████████████████████████████| 26.9MB 1.5MB/s 


In [ ]:
!nvidia-smi


Mon Apr 19 15:46:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## mount google drive and install repo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/GAN/stylegan2-ada-pytorch"):
    %cd "/content/drive/My Drive/GAN/stylegan2-ada-pytorch"
else:
    #install script
    %cd "/content/drive/My Drive/GAN/"
    !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

/content/drive/My Drive/GAN/stylegan2-ada-pytorch


In [ ]:
!ls

calc_metrics.py  docs		  metrics		       torch_utils
data		 downloads	  network-snapshot-008064.pkl  training
datasets	 download.sh	  out			       training-runs
dataset_tool.py  generate.py	  projector.py		       train.py
dnnlib		 latent_space.py  __pycache__
Dockerfile	 legacy.py	  README.md
docker_run.sh	 LICENSE.txt	  style_mixing.py


## Latent code research

In [ ]:
import os
import re
from typing import List, Optional

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy
def num_range(s: str) -> List[int]:
    '''Accept either a comma separated list of numbers 'a,b,c' or a range 'a-c' and return as a list of ints.'''

    range_re = re.compile(r'^(\d+)-(\d+)$')
    m = range_re.match(s)
    if m:
        return list(range(int(m.group(1)), int(m.group(2))+1))
    vals = s.split(',')
    return [int(x) for x in vals]

In [ ]:
def generate_images(network_pkl, seed, truncation_psi, outdir, noise_mode):

    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

    os.makedirs(outdir, exist_ok=True)

    # Labels.
    label = torch.zeros([1, G.c_dim], device=device)

    # Generate images.
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    print('laten shape', z.shape)  # 1, 512
    img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
    img = img[0].cpu().numpy()
    img = PIL.Image.fromarray(img)
    #print('img type', type(img))
    return seed, img, z

In [ ]:
def interpolate(z1, z2, rate):
    return z1*rate + z2*(1-rate)

In [ ]:
def generate_images_from_z(rate, network_pkl, z, truncation_psi, outdir, noise_mode):

    #print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

    os.makedirs(outdir, exist_ok=True)

    # Labels.
    label = torch.zeros([1, G.c_dim], device=device)

    # Generate images.
    z = z.to(device)
    #print('laten shape', z.shape)  # 1, 512
    img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/inter{rate}.png')
    img = img[0].cpu().numpy()
    img = PIL.Image.fromarray(img)
    #print('img type', type(img))
    return img, z

In [ ]:
#interpolation gif, visualizing the effect of interpolating from img1 to img2
def get_images(img1, img2):
  #print('img1', img1.shape)
  #height1, width1, _ = img1.shape
  #height2, width2, _ = img2.shape
  dst = PIL.Image.new('RGB', (img1.width + img2.width, img1.height))
  dst.paste(img1, (0, 0))
  dst.paste(img2, (img1.width, 0))
  return dst

def make_gif(img1, img2, z1, z2, nums):
  step_size = 1.0/nums
  total_img = []
  amounts = np.arange(0, 1, step_size)
  for rate in tqdm(amounts):
    inter_num = interpolate(z1, z1, rate)
    cur_img, _ = generate_images_from_z(network_pkl='https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl', z=inter_num, truncation_psi=1, outdir='./out', noise_mode='const')
    #print(cur_img.shape)
    frame = get_images(img1, cur_img)
    frame = get_images(frame, img2)
    total_img.append(frame)
  save_path = './out/test.gif'
  total_img[0].save(save_path, save_all=True, append_images=total_img[1:], duration=1000, loop=0)


##AFHQ Cat

In [ ]:
!python latent_space.py --outdir=out --trunc=1 --seeds=100\
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl"...
Generating image for seed 100 (0/1) ...
laten shape torch.Size([1, 512])
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [ ]:
# def generate_images(network_pkl, seed, truncation_psi, outdir):
_, img1, z1 = generate_images(seed=10, network_pkl='https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl', truncation_psi=1, outdir='./out', noise_mode='const')
_, img2, z2 = generate_images(seed=100, network_pkl='https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl', truncation_psi=1, outdir='./out', noise_mode='const')


Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl"...
laten shape torch.Size([1, 512])
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl"...
laten shape torch.Size([1, 512])


In [ ]:
# interpolation latent z
inter_z = interpolate(z1, z2, rate=0.95)
inter_z.shape

torch.Size([1, 512])

In [ ]:
img3 = generate_images_from_z(rate=0.95, network_pkl='https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl', z=inter_z,  truncation_psi=1, outdir='./out', noise_mode='const')


In [ ]:
from tqdm import tqdm
import cv2

make_gif(img1, img2, z1, z2, nums=10)

100%|██████████| 10/10 [00:04<00:00,  2.42it/s]


## Cow

In [ ]:
# def generate_images(network_pkl, seed, truncation_psi, outdir):
_, img11, z11 = generate_images(seed=10, network_pkl='./network-snapshot-008064.pkl', truncation_psi=1, outdir='./out', noise_mode='const')
_, img22, z22 = generate_images(seed=100, network_pkl='./network-snapshot-008064.pkl', truncation_psi=1, outdir='./out', noise_mode='const')


Loading networks from "./network-snapshot-008064.pkl"...
laten shape torch.Size([1, 512])
Loading networks from "./network-snapshot-008064.pkl"...
laten shape torch.Size([1, 512])


In [ ]:
inter_z = interpolate(z11, z22, rate=0.95)
inter_z.shape

torch.Size([1, 512])

In [ ]:
img33 = generate_images_from_z(rate=0.95, network_pkl='./network-snapshot-008064.pkl', z=inter_z,  truncation_psi=1, outdir='./out', noise_mode='const')
